In [ ]:
!pip3 install datasets
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

import math
import random
import numpy as np
import pickle
import nltk
import matplotlib.pyplot as plt
import csv
import time
import re

from datasets import load_dataset
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from datasets import load_dataset
from torch.utils.data import Dataset

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
## to do
# masking
# positional encoding
# dataset making ===> completed
#training

In [ ]:
class Multi30K(Dataset):
  def __init__(self,size,maxlen,batch_size,device,save):
    self.size   = size
    self.device = device
    self.save   = save
    self.maxlen = maxlen
    self.batch_size = batch_size
    self.dataset= self.initlization()
    self.english,self.dutch = self.DataPreprocessing()
    self.engTokens,self.dutTokens = self.DataTokenization()
    self.engVectors,self.dutVectors = self.DataVectroization()
    self.src_vocab = len(self.engVectors)
    self.trg_vocab = len(self.dutVectors)
    self.src_eos  = self.engVectors["<eos>"]
    self.src_sos  = self.engVectors["<sos>"]
    self.src_pad  = self.engVectors["<pad>"]
    self.trg_eos  = self.dutVectors["<eos>"]
    self.trg_sos  = self.dutVectors["<sos>"]
    self.trg_pad  = self.dutVectors["<pad>"]
    self.batchs   = self.batchEnurmation()

  def initlization(self):
    dataset = load_dataset("bentrevett/multi30k")
    dataset = dataset["train"]
    return dataset

  def preprocessing(self,text):
    patt = r'[^\w\s]'
    text = re.sub(patt,"",text)
    return text

  def DataPreprocessing(self):
    english = []
    dutch  = []
    a = time.time()
    if not self.save:
      for i in range(self.size):
        en = self.dataset[i]["en"].lower()
        en = self.preprocessing(en)
        de = self.dataset[i]["de"].lower()
        de = self.preprocessing(de)
        english.append(en)
        dutch.append(de)
      with open("Weights/EnglishPreprocessing","wb") as f:
        pickle.dump(english,f)
      with open("Weights/DutchPreprocessing","wb") as f:
        pickle.dump(dutch,f)
    if self.save:
      with open("Weights/EnglishPreprocessing","rb") as f:
        english = pickle.load(f)
      with open("Weights/DutchPreprocessing","rb") as f:
        dutch  = pickle.load(f)
    b = time.time()
    print(f'It took {b-a} seconds to preprocess the entrie Xsum dataset')
    return english,dutch

  def DataTokenization(self):
    a = time.time()
    english ,dutch = [],[]
    if not self.save:
      for i in range(self.size):
        en = word_tokenize(self.english[i])
        du = word_tokenize(self.dutch[i])
        english.append(en)
        dutch.append(du)

      with open("Weights/EnglishTokens","wb") as f:
        pickle.dump(english,f)

      with open("Weights/DutchTokens","wb") as f:
        pickle.dump(dutch,f)

    if self.save:
      with open("Weights/EnglishtTokens","rb") as f:
        english = pickle.load(f)

      with open("Weights/DutchdTokens","rb") as f:
        dutch = pickle.load(f)
    b = time.time()
    print(f"it took {b-a} seconds to tokenization the entrie XSum dataset")
    return english,dutch

  def DataVectroization(self):
    a = time.time()

    if not self.save:
      eng = Word2Vec(self.engTokens,min_count = 1)
      dut = Word2Vec(self.dutTokens,min_count = 1)
      eng = eng.wv.key_to_index
      dut = dut.wv.key_to_index
      eng.update({"<sos>":len(eng)})
      eng.update({"<eos>":len(eng)})
      eng.update({"<pad>":len(eng)})
      dut.update({"<sos>":len(dut)})
      dut.update({"<eos>":len(dut)})
      dut.update({"<pad>":len(dut)})
      with open("Weights/englishVectors","wb") as f:
        pickle.dump(eng,f)

      with open("Weights/dutchVectors","wb") as f:
        pickle.dump(dut,f)
    else:
      with open("Weights/englishVectors","rb") as f:
        eng = pickle.load(f)

      with open("Weights/dutchVectors","rb") as f:
        dut = pickle.load(f)

    b = time.time()
    print(f"it took {b-a} seconds to Vectorize the dataset")
    return eng,dut

  def vectorization(self,src,vectors):
    main = []
    for i in range(len(src)):
      main.append(vectors[src[i]])
    return main

  def tokenization(self,src,trg):
    src_len = len(src)
    trg_len = len(trg)

    if src_len > self.maxlen or trg_len > self.maxlen:
      src = torch.tensor([random.randint(0,self.src_vocab) for _ in range(self.maxlen)])
      trg = torch.tensor([random.randint(0,self.trg_vocab) for _ in range(self.maxlen)])
      print("sentence was too long")

    else:
      src_pad = self.maxlen - src_len -2
      # src = torch.cat([
      #     torch.tensor([self.src_sos]),
      #     src,
      #     torch.tensor([self.src_pad]*src_pad),
      #     torch.tensor([self.src_eos])
      # ]).to(self.device)
      src = [self.src_sos] + src + [self.src_pad]*src_pad + [self.src_eos]
      trg_pad = self.maxlen - trg_len - 2
      # trg = torch.cat([
      #     torch.tensor([self.trg_sos]),
      #     trg,
      #     torch.tensor([self.trg_pad]*trg_pad),
      #     torch.tensor([self.trg_eos])
      # ])
      trg = [self.trg_sos] + trg + [self.trg_pad]*trg_pad + [self.trg_eos]

    return src,trg

  def batchEnurmation(self):
    batchs = []
    for i in range(self.size):
      if i%25==0:
        batchs.append(i)
    return batchs

  def __len__(self):
    return len(self.batchs)

  def __getitem__(self,idx):
    src_main,trg_main = [],[]
    src_main = self.english[self.batchs[idx]:self.batchs[idx]+25]
    trg_main = self.dutch[self.batchs[idx]:self.batchs[idx]+25]
    src,trg = [],[]
    for i in range(self.batch_size):
      src_te = src_main[i]
      src_te = word_tokenize(src_te)
      src_te = self.vectorization(src_te,self.engVectors)
      trg_te = trg_main[i]
      trg_te = word_tokenize(trg_te)
      trg_te = self.vectorization(trg_te,self.dutVectors)
      src_te,trg_te = self.tokenization(src_te,trg_te)
      src.append(src_te)
      trg.append(trg_te)
    src = torch.tensor(src).int().to(self.device)
    trg = torch.tensor(trg).int().to(self.device)
    return src,trg

In [ ]:
class Embedding(nn.Module):
  def __init__(self,d_model,vocab,drop,device):
    super().__init__()
    self.d_model = d_model
    self.embed = nn.Embedding(vocab,d_model).to(device)
    self.drop  = nn.Dropout(drop)

  def forward(self,x):
    x = self.embed(x)
    return self.drop(x)/math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self,d_model,maxlen,device):
    super().__init__()
    self.pe = torch.zeros(maxlen,d_model).to(device)
    position = torch.arange(0,maxlen).float().unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    self.pe[:, 0::2] = torch.sin(position * div_term)
    self.pe[:, 1::2] = torch.cos(position * div_term)
    self.pe = self.pe.unsqueeze(0)
  def forward(self,x):
    x = x + (self.pe[:, :x.shape[0], :]).requires_grad_(False)
    return x.squeeze(0)

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_model,heads,device):
    super().__init__()
    self.d_model = d_model
    self.heads   = heads
    self.d_k     = d_model // heads
    self.w_q = nn.Linear(d_model,d_model).to(device)
    self.w_k = nn.Linear(d_model,d_model).to(device)
    self.w_v = nn.Linear(d_model,d_model).to(device)
    self.soft = nn.Softmax()

    self.w_out = nn.Linear(d_model,d_model).to(device)
  def attention(self,q,k,v,mask = None):
    scores = q @ k.transpose(2,3)
    #1. scaling
    scores = scores/ math.sqrt(self.d_k)
    print(scores.shape)
    #2. masking
    if mask is not None:
      scores.masked_fill_(mask,-10000)
    #3. applying softmax
    scores = self.soft(scores)
    return scores @ v #attention

  def split(self,ten):
    batch_size,length,d_model = ten.size()
    ten = ten.view(batch_size,self.heads,length,self.d_k).transpose(1, 2)
    return ten

  def concat(self,attention):
    attention = attention.transpose(1, 2).contiguous()
    batch_size,heads,seqlen,d_k = attention.size()
    attention = attention.view(batch_size,seqlen,self.d_model)
    return attention


  def forward(self,q,k,v,mask):
    q = self.w_q(q)
    k = self.w_k(k)
    v = self.w_v(v)

    q = self.split(q)
    k = self.split(k)
    v = self.split(v)

    attention = self.attention(q,k,v)
    attention = self.concat(attention)
    attention = self.w_out(attention)
    return attention

NameError: name 'nn' is not defined

In [ ]:
device = torch.device("cuda")
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model)).to(device)
        self.beta = nn.Parameter(torch.zeros(d_model)).to(device)
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased=False, keepdim=True)
        out = (x - mean) / torch.sqrt(var + self.eps)
        out = self.gamma * out + self.beta
        return out

In [ ]:
class FeedForward(nn.Module):
  def __init__(self,d_model,hidden,device,drop=0.1):
    super().__init__()
    self.layer1 = nn.Linear(d_model,hidden).to(device)
    self.layer2 = nn.Linear(hidden,d_model).to(device)
    self.drop   = nn.Dropout(drop)
    self.relu   = nn.ReLU()

  def forward(self,x):
    x = self.layer1(x)
    x = self.drop(self.relu(x))
    x = self.layer2(x)
    return x

In [ ]:
class EncoderBlock(nn.Module):
  def __init__(self,d_model,heads,hidden,device,drop = 0.1):
    super().__init__()
    self.attention = MultiHeadAttention(d_model,heads,device)
    self.norm1     = LayerNorm(d_model)
    self.norm2     = LayerNorm(d_model)
    self.network   = FeedForward(d_model,hidden,device)
    self.drop1     = nn.Dropout(drop)
    self.drop2     = nn.Dropout(drop)
  def forward(self,x,src_mask):
    attention      = self.attention(x,x,x,src_mask)
    attention      = self.drop1(attention)
    x              = self.norm1(x+attention)#residualConnection1

    feednetwork    = self.network(x)
    feednetwork    = self.drop2(feednetwork)
    x              = self.norm2(x+feednetwork)
    return x

In [ ]:
class Encoder(nn.Module):
  def __init__(self,layers):
    super().__init__()
    self.layers = layers

  def forward(self,x,src_mask):
    for layer in self.layers:
      x = layer(x,src_mask)
    return x


In [ ]:
class DecoderBlock(nn.Module):
  def __init__(self,d_model,heads,hidden,device,drop = 0.1):
    super().__init__()
    self.masked_attention = MultiHeadAttention(d_model,heads,device)
    self.norm1            = LayerNorm(d_model)
    self.drop1            = nn.Dropout(drop)

    self.multiAttention   = MultiHeadAttention(d_model,heads,device)
    self.norm2            = LayerNorm(d_model)
    self.drop2            = nn.Dropout(drop)

    self.network          = FeedForward(d_model,hidden,device)
    self.norm3            = LayerNorm(d_model)
    self.drop3            = nn.Dropout(drop)

  def forward(self,src,trg,src_mask,trg_mask):
    attention = self.masked_attention(trg,trg,trg,trg_mask)
    attention = self.drop1(attention)
    x         = self.norm1(trg+attention)

    attention = self.multiAttention(src,src,trg,src_mask)
    attention = self.drop2(attention)
    x         = self.norm2(x+attention)

    feednetwork    = self.network(x)
    feednetwork    = self.drop3(feednetwork)
    x              = self.norm3(x+feednetwork)

    return x

In [ ]:
class Decoder(nn.Module):
  def __init__(self,layers,d_model,vocab_size,device):
    super().__init__()
    self.layers = layers
    self.linear = nn.Linear(d_model,vocab_size).to(device)
    self.softmax = nn.Softmax()

  def forward(self,enc_out,x,src_mask,trg_mask):
    for layer in self.layers:
      x = layer(enc_out,x,src_mask,trg_mask)
    x = self.linear(x)
    x = self.softmax(x)
    return x

In [ ]:
class Transformer(nn.Module):
  def __init__(self,d_model,heads,src_vocab,trg_vocab,maxlen,hidden,layers,drop,src_mask,trg_mask,device,src_pad,trg_pad):
    super().__init__()
    self.d_model = d_model
    self.heads   = heads
    self.src_vocab = src_vocab
    self.trg_vocab = trg_vocab
    self.hidden    = hidden
    self.device    = device
    self.layers    = layers
    self.src_pad   = src_pad
    self.trg_pad   = trg_pad
    self.src_embedding = Embedding(d_model,src_vocab,drop,device)
    self.trg_embedding = Embedding(d_model,trg_vocab,drop,device)
    self.src_pos       = PositionalEncoding(d_model,maxlen,device)
    self.trg_pos       = PositionalEncoding(d_model,maxlen,device)
    self.encoder       = self.encoderMaker()
    self.decoder       = self.decoderMaker()
    self.src_mask      = src_mask
    self.trg_mask      = trg_mask
    self.__initweights__()

  def encoderMaker(self):
    encoder = []
    for i in range(self.layers):
      encblock = EncoderBlock(self.d_model,self.heads,self.hidden,self.device)
      encoder.append(encblock)
    m_encoder = Encoder(encoder)
    return m_encoder

  def decoderMaker(self):
    decoder = []
    for i in range(self.layers):
      decblock = DecoderBlock(self.d_model,self.heads,self.hidden,self.device)
      decoder.append(decblock)
    m_decoder = Decoder(decoder,self.d_model,self.trg_vocab,self.device)
    return m_decoder

  def srcMasking(self,src):
    src_mask = (src != self.src_pad).unsqueeze(1).unsqueeze(2)
    return src_mask.view(src.shape[0],src.shape[1])

  def trgcMasking(self,trg):
    trg_mask = (trg != self.trg_pad).unsqueeze(1).unsqueeze(2)
    return trg_mask.view(trg.shape[0],trg.shape[1])

  def forward(self,src,trg):
    src_mask = self.srcMasking(src)
    src = self.src_embedding(src)
    #src = self.src_pos(src)
    trg_mask = self.trgcMasking(trg)
    trg = self.trg_embedding(trg)
    #trg = self.trg_pos(trg)

    enc_out = self.encoder(src,src_mask)
    dec_out = self.decoder(enc_out,trg,src_mask,trg_mask)
    return dec_out

  def __initweights__(self):
    for m in self.modules():
      if isinstance(m,nn.Linear):
        nn.init.kaiming_normal(m.weight)

      elif isinstance(m,nn.Embedding):
        nn.init.kaiming_normal(m.weight)


In [ ]:
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = Multi30K(29000,42,25,device,False)
d_model = 512
src_vocab = data.src_vocab
trg_vocab = data.trg_vocab
maxlen    = 42
hidden    = 1024*2
heads     = 8
layers    = 6
drop      = 0.1
src_mask  = 0.00005
trg_mask  = 0.0008
transformer = Transformer(d_model,heads,src_vocab,trg_vocab,maxlen,hidden,layers,drop,src_mask,trg_mask,device,data.src_pad,data.trg_pad)
optimizer   = optim.Adam(transformer.parameters(),lr = 1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 verbose=True,
                                                 factor=0.9,
                                                 patience=10)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


It took 3.8282346725463867 seconds to preprocess the entrie Xsum dataset
it took 16.025455474853516 seconds to tokenization the entrie XSum dataset
it took 8.886495351791382 seconds to Vectorize the dataset


<ipython-input-13-bbac73ec86c1>:65: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight)
<ipython-input-13-bbac73ec86c1>:62: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight)


In [ ]:
src = []
for i in range(25):
  src.append([random.randint(0,499) for _ in range(42)])
src = torch.tensor(src)

trg = []
for i in range(25):
  trg.append([random.randint(0,499) for _ in range(42)])
trg = torch.tensor(src)
print(src.shape)

torch.Size([25, 42])


<ipython-input-15-3816304e3ae3>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trg = torch.tensor(src)


In [ ]:
class Main():
  def __init__(self,max_size,ignore_index,clip):
    self.maxsize = max_size
    self.ignore_idx = ignore_index
    self.clip = clip
    self.loss = nn.CrossEntropyLoss()
    self.epochs = []
    self.losses = []

  def argmax(self,src):
    main = []
    for i in range(src.shape[0]):
      main.append(src[i].argmax(1).tolist())
    return torch.tensor(main).to(device)

  def lossMasking(self,trg,output):
    mask = trg != self.ignore_idx
    mask = mask.to(device)
    trg = trg[mask]
    output = output[mask]
    loss   = self.loss(output.float(),trg.float())
    loss   = loss / 1.0e+9

    return torch.tensor([loss]).to(device)

  def ploting(self):
    plt.plot(self.epochs,self.losses)
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.savefig("Weights/Transformer.png")
    plt.close()

  def SaveandLoad(self,idx):
    if idx % 100 ==0:
      torch.save(transformer.state_dict(),"Weights/Tranaformer.pth")

    elif idx % 250 == 0:
      transformer.load_state_dict(torch.load("Weights/Tranaformer.pth"))

  def train(self):
    transformer.load_state_dict(torch.load("/content/Weights/Tranaformer.pth"))
    transformer.train()
    for i in range(self.maxsize):
      src,trg = data[i]
      output  = transformer(src,trg)
      output  = self.argmax(output)
      loss    = self.lossMasking(trg,output)
      loss.requires_grad = True
      optimizer.zero_grad()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(transformer.parameters(), self.clip)
      optimizer.step()
      self.losses.append(loss.item())
      self.epochs.append(i)
      self.ploting()
      self.SaveandLoad(i)
      print(f"epochs : {i}, loss : {loss}, completion : {round(i/self.maxsize,3)}%")

  def eval(self,data):
    transformer.eval()
    with torch.no_grad():
      for i in range(len(data)):
        src,trg = data[i]
        output  = transformer(src,trg)
        output  = self.argmax(output)


In [ ]:
if __name__ =="__main__":
  main = Main(1160,data.trg_pad,1)
  main.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epochs : 0, loss : tensor([9.9244], device='cuda:0', requires_grad=True), completion : 0.0%
epochs : 1, loss : tensor([10.8151], device='cuda:0', requires_grad=True), completion : 0.001%
epochs : 2, loss : tensor([9.2701], device='cuda:0', requires_grad=True), completion : 0.002%
epochs : 3, loss : tensor([9.7544], device='cuda:0', requires_grad=True), completion : 0.003%
epochs : 4, loss : tensor([9.0920], device='cuda:0', requires_grad=True), completion : 0.003%
epochs : 5, loss : tensor([9.3475], device='cuda:0', requires_grad=True), completion : 0.004%
epochs : 6, loss : tensor([10.9716], device='cuda:0', requires_grad=True), completion : 0.005%
epochs : 7, loss : tensor([10.9478], device='cuda:0', requires_grad=True), completion : 0.006%
epochs : 8, loss : tensor([9.6437], device='cuda:0', requires_grad=True), completion : 0.007%
epochs : 9, loss : tensor([9.7555], device='cuda:0', requires_grad=True), completion : 0.008%
epochs : 10, loss : tensor([11.2765], device='cuda:0', requ

In [ ]:
a = torch.tensor([13198, 10435, 12141, 14180,  6598, 16240, 16721,  6137, 17302, 10244,
         8315,  6223, 10668, 14984,  7417, 10081,  4252,  6384, 17491, 16338,
        10061,  7807,  7179,  6232,  2653, 13462,  9298,  6705,  9274, 17436,
         8540, 11409,  5310,   696,  2666,  5318,  7463,  6819,  1942, 15402,
        17445,  1005])

In [ ]:
data.batch_size

25